In [1]:
import pandas as pd
from IPython.core.display import HTML
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
all_league = pd.read_csv(r'D:/Documents/sheng_final_project/Visualization/ETL-EDA/2020_LoL_esports_match_data_from_OraclesElixir_20201124.csv')
lcs = all_league[all_league.league == 'LCS']

C:\Users\ishengy\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
lcs = lcs.astype({"year": str, 
                 "playoffs": bool, 
                 "game": str, 
                 "patch": str, 
                 "playerid": str,
                 "result": bool,
                 "firstdragon": bool,
                 "firstherald": bool,
                 "firstbaron": bool,
                 "firstmidtower": bool,
                 "firsttothreetowers": bool,
                 "firsttower": bool,
                 "firstblood": bool,
                 "firstbloodassist": bool,
                 "firstbloodkill": bool,
                 "firstbloodvictim": bool})

In [4]:
descript = lcs.describe().T.drop('count',axis=1)
display(descript)
#descript.to_excel('descript.xlsx')

,mean,std,min,25%,50%,75%,max
gamelength,2062.306818,388.440423,1337.0,1793.25,1988.5,2292.25,3680.0
kills,3.799874,4.701951,0.0,1.00,2.0,5.00,30.0
deaths,3.803030,4.513989,0.0,1.00,2.5,4.00,30.0
assists,9.107323,11.105225,0.0,3.00,6.0,10.00,86.0
teamkills,11.399621,6.139009,0.0,6.00,11.0,16.00,30.0
...,...,...,...,...,...,...,...
opp_xpat15,9687.400253,8786.186100,2681.0,4889.00,6321.5,7720.25,33530.0
opp_csat15,169.661616,158.045564,0.0,90.00,126.0,149.00,594.0
golddiffat15,0.000000,1278.069747,-7782.0,-560.00,0.0,560.00,7782.0
xpdiffat15,0.000000,974.875182,-5909.0,-497.50,0.0,497.50,5909.0


In [5]:
data_type = lcs.dtypes
print(data_type)

gameid               object
datacompleteness     object
url                  object
league               object
year                 object
                     ...   
opp_xpat15          float64
opp_csat15          float64
golddiffat15        float64
xpdiffat15          float64
csdiffat15          float64
Length: 105, dtype: object


In [6]:
lcs_percent_missing = lcs.isnull().sum() * 100 / len(lcs)
lcs_percent_missing = lcs_percent_missing[lcs_percent_missing!=0]
display(lcs_percent_missing.sort_values(ascending = False))

dragons (type unknown)    99.684343
opp_elders                83.648990
opp_elementaldrakes       83.648990
infernals                 83.648990
mountains                 83.648990
clouds                    83.648990
oceans                    83.648990
elders                    83.648990
elementaldrakes           83.648990
opp_dragons               83.333333
opp_heralds               83.333333
barons                    83.333333
gspd                      83.333333
opp_barons                83.333333
towers                    83.333333
opp_towers                83.333333
dragons                   83.333333
heralds                   83.333333
champion                  16.666667
total cs                  16.666667
damageshare               16.666667
earnedgoldshare           16.666667
player                    16.666667
ban5                       0.378788
ban3                       0.189394
dtype: float64

## Explanation of Missing Data and Solutions:

**Explanation:** 0.38% of ban5 and 0.19% of ban3 are missing because of penalties or timeouts: Teams only have 2 rounds of 30 seconds each to ban out certain champions. Ban3 occurs in the first round of bans and ban5 occurs in the second round of bans. Teams can be penalized with a loss of ban for violations such as not following code of conduct.

**Solution:** Fill in missing data with "None"

In [7]:
lcs.fillna({'ban3': 'None', 'ban5': 'None'}, inplace=True)

**Explanation:** 99.68% of Dragons (type unknown) is missing. Looking at the dataset, this variable is only filled in when the dragon-type columns Infernals, Mountains, Clouds, Oceans, Elders are empty. This could be an error in data collection from the tournament realm since all dragon-types are visible by nature of the game. The occurance is also only about 0.32%.

**Solution:** Unfortunately, because the data wasn't properly collected, there is no way to split "Dragons (Type Unknown)" between "elementaldrakes" and "opp_elementaldrakes" (aka: team dragons versus opponent dragons). "dragons" and "Opp_dragons" are suitable replacements. In order to keep the datatype consistent throughout the columns, the data will be filled in as 0, but the rows will be filtered out in the visualization when looking at individual elemental dragon objectives since the rest of the data is still required.

In [8]:
lcs.fillna({'elementaldrakes': 0,
            'opp_elementaldrakes': 0, 
            'opp_elementaldrakes': 0,
            'infernals': 0,
            'mountains': 0, 
            'clouds': 0, 
            'oceans': 0, 
            'elders': 0, 
            'opp_elders': 0}, inplace=True)

**Explanation:** 16.67% of the data is missing Player name - this is probably because the data includes aggregated team statistics as well, which are null in the Player column. The other stats that are also missing 16.67% of the data are only available on a team-level granularity, not on a smaller player-level granularity. Similarly, variables such as dragons, heralds, towers, and other map objects are missing 83.33% of the the data. This data is only available on a Player-level granularity, not on a higher team-level granularity. As was previously noted, approximately 0.32% of the missing data is contributed by the Dragons (Type Unknown) error.

**Solution:** Because of expected interchangable use-cases, the player and team data will remain as one dataset since *Team* data can be filtered out on *position != 'team'*. 

## Final Dataset Status:

In [9]:
player_percent_missing = lcs.isnull().sum() * 100 / len(lcs)
player_percent_missing = player_percent_missing[player_percent_missing!=0]
display(player_percent_missing.sort_values(ascending = False))

dragons (type unknown)    99.684343
gspd                      83.333333
opp_towers                83.333333
towers                    83.333333
opp_barons                83.333333
barons                    83.333333
opp_heralds               83.333333
heralds                   83.333333
opp_dragons               83.333333
dragons                   83.333333
total cs                  16.666667
earnedgoldshare           16.666667
damageshare               16.666667
champion                  16.666667
player                    16.666667
dtype: float64

## Output Dataset

In [10]:
lcs.to_excel(r'D:/Documents/sheng_final_project/Visualization/ETL-EDA/lcs_dataset.xlsx', sheet_name='data', index = False)

## Note on Additional ETL in Tableau:
Table was created based on output to pivot data for team bans 1-5 within Tableau